In [2]:
from mainnet_launch.pages.risk_metrics.incentive_token_prices_actual_vs_expected import _load_sales_df
import plotly.express as px

df = _load_sales_df()

In [8]:
df.columns

Index(['tx_hash', 'log_index', 'chain_id', 'datetime', 'third_party_price',
       'sell_amount', 'buy_amount', 'buy_amount_received', 'sell', 'buy',
       'actual_execution', 'worst_possible_execution', 'label',
       'price_diff_pct'],
      dtype='object')

In [13]:
df

,tx_hash,log_index,chain_id,datetime,third_party_price,sell_amount,buy_amount,buy_amount_received,sell,buy,actual_execution,worst_possible_execution,label,price_diff_pct
0,0x565ad90b76a507025fd4091d1d7103c1561e98ac2278...,91,1,2025-04-11 12:08:11+00:00,0.614746,507.439671,309.018517,311.196896,CRV,USDC,0.613269,0.608976,CRV -> USDC,-0.240301
1,0x565ad90b76a507025fd4091d1d7103c1561e98ac2278...,53,1,2025-04-11 12:08:11+00:00,2.169004,1.014879,2.181971,2.197353,CVX,USDC,2.165137,2.149981,CVX -> USDC,-0.178276
2,0x32958cb912580f139f042a8e8d2fa1978fb08deb7c40...,92,1,2025-07-20 07:01:35+00:00,0.000008,6021.122074,0.047298,0.047631,SWISE,WETH,0.000008,0.000008,SWISE -> WETH,-0.003472
3,0x32958cb912580f139f042a8e8d2fa1978fb08deb7c40...,4,1,2025-07-20 07:01:35+00:00,0.000274,176.657077,0.047948,0.048286,DAI,WETH,0.000273,0.000271,DAI -> WETH,-0.120665
4,0x32958cb912580f139f042a8e8d2fa1978fb08deb7c40...,58,1,2025-07-20 07:01:35+00:00,0.000349,45.619031,0.015817,0.015929,BAL,WETH,0.000349,0.000347,BAL -> WETH,0.078363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8702,0xfbba87e4ef6b52bd16f19852baa0d84231748ea901bb...,805,8453,2025-09-17 18:55:35+00:00,0.000040,55.555742,0.002185,0.002201,AURA,WETH,0.000040,0.000039,AURA -> WETH,-0.642081
8703,0xfbba87e4ef6b52bd16f19852baa0d84231748ea901bb...,788,8453,2025-09-17 18:55:35+00:00,0.000286,9.279310,0.002626,0.002644,AERO,WETH,0.000285,0.000283,AERO -> WETH,-0.267120
8704,0x8969cf676fece0f3afa3f1e72955a6815fecd4f47135...,56,8453,2025-09-17 18:56:11+00:00,0.999785,352.264827,349.684950,352.159907,GHO,USDC,0.999702,0.992676,GHO -> USDC,-0.008286
8705,0x8969cf676fece0f3afa3f1e72955a6815fecd4f47135...,19,8453,2025-09-17 18:56:11+00:00,5.281523,24.667059,131.592360,132.521810,FLUID,USDC,5.372420,5.334741,FLUID -> USDC,1.721046


In [ ]:
import plotly.io as pio

pio.templates.default = "plotly_white"